In [2]:
import sys

sys.path.insert(0, '../Lab/')
sys.path.insert(0, '../New_Hybrids/')
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from UserIcmKNNCFRecommender import UserIcmKNNCFRecommender
from ItemIcmKNNCFRecommender import ItemIcmKNNCFRecommender
from WeightedHybridV2 import WeightedHybridScoreRecommender
from optimize_weights import optimize_weights
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
import matplotlib
import matplotlib.pyplot as plt
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python

%matplotlib inline 

import numpy as np
sys.path.insert(0, '../datasets/')
from load_tables import load_ICM_csr, load_URM_csr, load_clustered_ICM_csr
def plotta(t,maps):
    fig, ax = plt.subplots()
    ax.plot(t, maps)

    ax.set(xlabel='T', ylabel='MAP')
    ax.grid()
    plt.show()

In [3]:
ICM_all = load_ICM_csr()
ICM_cl = load_clustered_ICM_csr()
URM_all = load_URM_csr()

In [4]:

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.9)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[ 10])

In [5]:
from FeatureWeighting.CFW_D_Similarity_Linalg import CFW_D_Similarity_Linalg

In [6]:
#    def fit(self, show_max_performance = False, logFile = None, loss_tolerance = 1e-6,
     #       iteration_limit = 50000, damp_coeff=0.0, topK = 300, add_zeros_quota = 0.0, normalize_similarity = False):

In [7]:
# def __init__(self, URM_train, ICM, S_matrix_target)

In [8]:

from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical

In [18]:
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from ItemIcmKNNCFRecommender import ItemIcmKNNCFRecommender


iicm = {}
iicm["topK"] = 141
iicm["shrink"] = 47

iknn = ItemIcmKNNCFRecommender(URM_train, ICM_all)
iknn.fit(**iicm)


ItemKNNCFRecommender: URM Detected 128 (1.61 %) cold users.
ItemKNNCFRecommender: URM Detected 3154 (12.14 %) cold items.
ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
ItemKNNCFRecommender: URM Detected 25 (0.10 %) cold items.
Similarity column 25975 ( 100 % ), 3015.67 column/sec, elapsed time 0.14 min
Similarity column 25975 ( 100 % ), 1931.02 column/sec, elapsed time 0.22 min


In [19]:
iknn.W_sparse

<25975x25975 sparse matrix of type '<class 'numpy.float32'>'
	with 3636610 stored elements in Compressed Sparse Row format>

In [20]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs


def hyperparameter_optimization_CFW_D(URM_train, ICM_train, W_sparse_CF):

    recommender_class = CFW_D_Similarity_Linalg

    parameterSearch = SearchBayesianSkopt(recommender_class,
                                     evaluator_validation=evaluator_validation,
                                     evaluator_test=evaluator_test)


    hyperparameters_range_dictionary = {}
    hyperparameters_range_dictionary["topK"] = Integer(5, 1000)
    hyperparameters_range_dictionary["add_zeros_quota"] = Real(low = 0, high = 1, prior = 'uniform')
    hyperparameters_range_dictionary["normalize_similarity"] = Categorical([True, False])


    recommender_input_args = SearchInputRecommenderArgs(
        CONSTRUCTOR_POSITIONAL_ARGS = [URM_train, ICM_train, W_sparse_CF],
        CONSTRUCTOR_KEYWORD_ARGS = {},
        FIT_POSITIONAL_ARGS = [],
        FIT_KEYWORD_ARGS = {}
    )


    output_folder_path = "result_experiments/"

    import os

    # If directory does not exist, create
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    n_cases = 100
    metric_to_optimize = "MAP"


    # Clone data structure to perform the fitting with the best hyperparameters on train + validation data
    recommender_input_args_last_test = recommender_input_args.copy()
    recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS[0] = URM_train + URM_validation


    parameterSearch.search(recommender_input_args,
                           recommender_input_args_last_test = recommender_input_args_last_test,
                           parameter_search_space = hyperparameters_range_dictionary,
                           n_cases = n_cases,
                           n_random_starts = int(n_cases/3),
                           save_model = "no",
                           output_folder_path = output_folder_path,
                           output_file_name_root = recommender_class.RECOMMENDER_NAME + "iknn",
                           metric_to_optimize = metric_to_optimize
                          )

In [ ]:
hyperparameter_optimization_CFW_D(URM_train, ICM_all, iknn.W_sparse)

Iteration No: 1 started. Evaluating function at random point.
CFW_D_Similarity_Linalg: URM Detected 128 (1.61 %) cold users.
CFW_D_Similarity_Linalg: URM Detected 3154 (12.14 %) cold items.
SearchBayesianSkopt: Testing config: {'topK': 692, 'add_zeros_quota': 0.14535899999828908, 'normalize_similarity': False}
CFW_D_Similarity_Linalg: Generating train data
Similarity column 25975 ( 100 % ), 1711.16 column/sec, elapsed time 0.25 min
CFW_D_Similarity_Linalg: Collaborative S density: 5.39E-03, nonzero cells 3636610
CFW_D_Similarity_Linalg: Content S density: 2.59E-02, nonzero cells 17469711
CFW_D_Similarity_Linalg: Generating train data. Sample 292526 ( 1.92 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 970963 ( 6.37 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 1954346 ( 12.81 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 3008464 ( 19.72 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 4075054 ( 26.72 %) 
CFW_D_Similarity_Linalg: Generatin

CFW_D_Similarity_Linalg: Generating train data. Sample 5003486 ( 32.95 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 6322378 ( 41.63 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 7889096 ( 51.95 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 9555244 ( 62.92 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 11205669 ( 73.78 %) 
CFW_D_Similarity_Linalg: Content S structure has 3636610 out of 22459644 ( 16.19%) nonzero collaborative cells
CFW_D_Similarity_Linalg: Nonzero collaborative cell sum is: 9.09E+04, average is: 2.50E-02, average over all collaborative data is 6.44E-03
Similarity column 25975 ( 100 % ), 1492.81 column/sec, elapsed time 0.29 min
EvaluatorHoldout: Processed 5464 ( 100.00% ) in 9.20 sec. Users per second: 594
SearchBayesianSkopt: Config 3 is suboptimal. Config: {'topK': 898, 'add_zeros_quota': 0.4788500215474596, 'normalize_similarity': True} - results: ROC_AUC: 0.0642121, PRECISION: 0.0130124, PRECISION_RECALL_MIN_DEN: 0

Similarity column 25975 ( 100 % ), 2495.30 column/sec, elapsed time 0.17 min
EvaluatorHoldout: Processed 5464 ( 100.00% ) in 6.55 sec. Users per second: 834
SearchBayesianSkopt: Config 6 is suboptimal. Config: {'topK': 226, 'add_zeros_quota': 0.6837012919817577, 'normalize_similarity': False} - results: ROC_AUC: 0.0696875, PRECISION: 0.0136896, PRECISION_RECALL_MIN_DEN: 0.0477062, RECALL: 0.0464131, MAP: 0.0223876, MRR: 0.0497778, NDCG: 0.0343465, F1: 0.0211430, HIT_RATE: 0.1368960, ARHR: 0.0545807, NOVELTY: 0.0051721, AVERAGE_POPULARITY: 0.0145314, DIVERSITY_MEAN_INTER_LIST: 0.9971786, DIVERSITY_HERFINDAHL: 0.9996996, COVERAGE_ITEM: 0.5198460, COVERAGE_ITEM_CORRECT: 0.0262175, COVERAGE_USER: 0.6875551, COVERAGE_USER_CORRECT: 0.0773877, DIVERSITY_GINI: 0.2447959, SHANNON_ENTROPY: 12.8609247, 

Iteration No: 7 ended. Evaluation done at random point.
Time taken: 516.0878
Function value obtained: -0.0224
Current minimum: -0.0229
Iteration No: 8 started. Evaluating function at random point

CFW_D_Similarity_Linalg: Generating train data. Sample 377717 ( 17.93 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 991730 ( 47.07 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 1603883 ( 76.13 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 2225299 ( 105.63 %) 
CFW_D_Similarity_Linalg: Content S structure has 2479136 out of 2480296 ( 99.95%) nonzero collaborative cells
CFW_D_Similarity_Linalg: Nonzero collaborative cell sum is: 1.89E+04, average is: 7.64E-03, average over all collaborative data is 6.44E-03
Similarity column 25975 ( 100 % ), 2641.10 column/sec, elapsed time 0.16 min
EvaluatorHoldout: Processed 5464 ( 100.00% ) in 6.42 sec. Users per second: 850
SearchBayesianSkopt: Config 10 is suboptimal. Config: {'topK': 96, 'add_zeros_quota': 0.17731592095810203, 'normalize_similarity': False} - results: ROC_AUC: 0.0684168, PRECISION: 0.0137811, PRECISION_RECALL_MIN_DEN: 0.0488077, RECALL: 0.0474913, MAP: 0.0220864, MRR: 0.0485357, NDCG: 0.0341970

CFW_D_Similarity_Linalg: URM Detected 128 (1.61 %) cold users.
CFW_D_Similarity_Linalg: URM Detected 3154 (12.14 %) cold items.
SearchBayesianSkopt: Testing config: {'topK': 543, 'add_zeros_quota': 0.7873158714534264, 'normalize_similarity': False}
CFW_D_Similarity_Linalg: Generating train data
Similarity column 25975 ( 100 % ), 2059.28 column/sec, elapsed time 0.21 min
CFW_D_Similarity_Linalg: Collaborative S density: 5.39E-03, nonzero cells 3636610
CFW_D_Similarity_Linalg: Content S density: 2.04E-02, nonzero cells 13793140
CFW_D_Similarity_Linalg: Generating train data. Sample 869547 ( 11.27 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 2571831 ( 33.33 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 4322173 ( 56.01 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 6101824 ( 79.07 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 7869666 ( 101.98 %) 
CFW_D_Similarity_Linalg: Generating train data. Sample 9608518 ( 124.51 %) 
CFW_D_Similarity_

In [ ]:
#Config 14: {'topK': 52, 'add_zeros_quota': 0.8588004964327149, 'normalize_similarity': False} itemknn

In [ ]:
# Config 9: {'topK': 128, 'add_zeros_quota': 0.4586460222330323, 'normalize_similarity': False} 
#     - results: ROC_AUC: 0.0688511, PRECISION: 0.0147694, PRECISION_RECALL_MIN_DEN: 0.0545477, 
#                     RECALL: 0.0529711, MAP: 0.0200068  itemknn